In [1]:
val sc = spark.sparkContext

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
3,application_1607211657348_0005,spark,idle,Link,Link


SparkSession available as 'spark'.
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@7f41f479


In [2]:
import org.apache.spark.sql.types.{FloatType,LongType}
import org.apache.spark.sql.types.{ArrayType, StructField, StructType}
import org.apache.spark.sql.functions.explode
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.types.{FloatType, LongType}
import org.apache.spark.sql.types.{ArrayType, StructField, StructType}
import org.apache.spark.sql.functions.explode
import org.apache.spark.sql.expressions.Window


In [3]:
import com.logicalclocks.hsfs._

import com.logicalclocks.hsfs._


In [4]:
val connection = HopsworksConnection.builder().build();
val fs = connection.getFeatureStore();

connection: com.logicalclocks.hsfs.HopsworksConnection = com.logicalclocks.hsfs.HopsworksConnection@5ae6638a
fs: com.logicalclocks.hsfs.FeatureStore = FeatureStore{id=129, name='amlsim2_featurestore', projectId=181, featureGroupApi=com.logicalclocks.hsfs.metadata.FeatureGroupApi@432a09d3}


In [5]:
val df_all = fs.getFeatureGroup("simts_features", 1).read()

df_all: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [f_333: double, f_329: double ... 364 more fields]


In [6]:
df_all.count()

res1: Long = 11000


In [7]:
df_all.where($"target" === 1).count()

res2: Long = 1000


In [8]:
df_all.where($"target" === 0).count()

res3: Long = 10000


In [9]:
print(df_all.columns.toSeq)

WrappedArray(f_333, f_329, f_328, f_327, f_320, f_312, f_307, f_302, f_301, f_299, f_296, f_293, f_286, f_285, f_281, f_280, f_277, f_266, f_261, f_256, f_255, f_254, f_253, f_252, f_251, f_249, f_240, f_235, f_232, f_230, f_228, f_226, f_222, f_215, f_211, f_209, f_207, f_193, f_186, f_185, f_177, f_175, f_173, f_172, f_170, f_169, f_163, f_162, f_158, f_155, f_153, f_152, f_151, f_146, f_142, f_141, f_140, f_138, f_132, f_130, f_127, f_124, f_117, f_114, f_111, f_110, f_106, f_104, f_103, f_102, f_100, f_97, f_94, f_84, f_80, f_79, f_78, f_77, f_65, f_62, f_60, f_59, f_55, f_50, f_49, f_46, f_41, f_40, f_34, f_32, f_24, f_23, f_17, f_14, f_11, f_10, f_8, f_4, f_3, f_334, f_341, f_348, f_350, f_358, f_365, f_323, f_321, f_317, f_311, f_297, f_295, f_292, f_287, f_276, f_270, f_269, f_267, f_265, f_258, f_242, f_233, f_229, f_217, f_216, f_208, f_205, f_204, f_203, f_202, f_201, f_200, f_196, f_195, f_191, f_184, f_183, f_182, f_181, f_179, f_174, f_167, f_164, f_159, f_156, f_154, f_1

In [10]:
print(df_all.columns.slice(0,365).map(a => a.toString ).toSeq)

WrappedArray(f_333, f_329, f_328, f_327, f_320, f_312, f_307, f_302, f_301, f_299, f_296, f_293, f_286, f_285, f_281, f_280, f_277, f_266, f_261, f_256, f_255, f_254, f_253, f_252, f_251, f_249, f_240, f_235, f_232, f_230, f_228, f_226, f_222, f_215, f_211, f_209, f_207, f_193, f_186, f_185, f_177, f_175, f_173, f_172, f_170, f_169, f_163, f_162, f_158, f_155, f_153, f_152, f_151, f_146, f_142, f_141, f_140, f_138, f_132, f_130, f_127, f_124, f_117, f_114, f_111, f_110, f_106, f_104, f_103, f_102, f_100, f_97, f_94, f_84, f_80, f_79, f_78, f_77, f_65, f_62, f_60, f_59, f_55, f_50, f_49, f_46, f_41, f_40, f_34, f_32, f_24, f_23, f_17, f_14, f_11, f_10, f_8, f_4, f_3, f_334, f_341, f_348, f_350, f_358, f_365, f_323, f_321, f_317, f_311, f_297, f_295, f_292, f_287, f_276, f_270, f_269, f_267, f_265, f_258, f_242, f_233, f_229, f_217, f_216, f_208, f_205, f_204, f_203, f_202, f_201, f_200, f_196, f_195, f_191, f_184, f_183, f_182, f_181, f_179, f_174, f_167, f_164, f_159, f_156, f_154, f_1

In [11]:
import org.apache.spark.ml.feature.{MinMaxScaler,StandardScaler,Normalizer, MaxAbsScaler}
import org.apache.spark.ml.{Pipeline}
import org.apache.spark.ml.feature.VectorAssembler

val assembler = new VectorAssembler().setInputCols(df_all.columns.slice(0,365).map(a => a.toString).toArray).setOutputCol("features")

val df_all_features = assembler.transform(df_all)

// scaler instance with the min(-1) and max(1) 
val mms = new MinMaxScaler().
    setInputCol("features").
    setOutputCol("mms").
    setMax(1).
    setMin(-1)

// scaler instance with the min(0) and max(1) 
val mms01 = new MinMaxScaler().
    setInputCol("features").
    setOutputCol("mms").
    setMax(1).
    setMin(0)

val stds = new StandardScaler().
    setInputCol("features").
    setOutputCol("stds")

val maxabsscaler = new MaxAbsScaler().
    setInputCol("features").
    setOutputCol("maxabsscaler")

val mms_tr = mms.setInputCol("features").setOutputCol("mms")
val mms01_tr = mms01.setInputCol("features").setOutputCol("mms01")
val mms_abs_tr = maxabsscaler.setInputCol("features").setOutputCol("maxabsscaler")
val stds_tr = stds.setInputCol("features").setOutputCol("stds")

val trainingFeaturesPipeline = (new Pipeline()
  .setStages(Array(mms_tr,mms01_tr,mms_abs_tr,stds_tr)))
val trainingFeaturesDF = trainingFeaturesPipeline.fit(df_all_features).transform(df_all_features)

import org.apache.spark.ml.feature.{MinMaxScaler, StandardScaler, Normalizer, MaxAbsScaler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.VectorAssembler
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_4e292fce9fa6
df_all_features: org.apache.spark.sql.DataFrame = [f_333: double, f_329: double ... 365 more fields]
mms: org.apache.spark.ml.feature.MinMaxScaler = minMaxScal_d1c43ebf334d
mms01: org.apache.spark.ml.feature.MinMaxScaler = minMaxScal_4c3208d654f5
stds: org.apache.spark.ml.feature.StandardScaler = stdScal_ab92a94378f8
maxabsscaler: org.apache.spark.ml.feature.MaxAbsScaler = maxAbsScal_e56c945c627c
mms_tr: mms.type = minMaxScal_d1c43ebf334d
mms01_tr: mms01.type = minMaxScal_4c3208d654f5
mms_abs_tr: maxabsscaler.type = maxAbsScal_e56c945c627c
stds_tr: stds.type = stdScal_ab92a94378f8
trainingFeaturesPipeline: org.apache.spark.ml.Pipeline = pipeline_8d2e9b9342ce
trainingFeaturesDF: org.apache.spark.sql.DataFrame = [f_333: double, f_

In [12]:
import org.apache.spark.ml.linalg.DenseVector
val toArr: Any => Array[Double] = _.asInstanceOf[DenseVector].toArray
val toArrUdf = udf(toArr)
val final_df_all = trainingFeaturesDF.select($"target",$"features",$"mms",$"mms01",$"maxabsscaler",$"stds").
             withColumn("target",$"target".cast(FloatType)).
             withColumn("features",toArrUdf($"features").cast(ArrayType(FloatType))).
             withColumn("mms",toArrUdf($"mms").cast(ArrayType(FloatType))).
             withColumn("mms01",toArrUdf($"mms01").cast(ArrayType(FloatType))).
             withColumn("maxabsscaler",toArrUdf($"maxabsscaler").cast(ArrayType(FloatType))).
             withColumn("stds",toArrUdf($"stds").cast(ArrayType(FloatType)))

import org.apache.spark.ml.linalg.DenseVector
toArr: Any => Array[Double] = <function1>
toArrUdf: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)
final_df_all: org.apache.spark.sql.DataFrame = [target: float, features: array<float> ... 4 more fields]


In [13]:
val van = final_df_all.where($"target"===1)
val ben = final_df_all.where($"target"===0) 

van: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: float, features: array<float> ... 4 more fields]
ben: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: float, features: array<float> ... 4 more fields]


In [14]:
// Now that the data has been prepared, let's split the dataset into a training and test dataframe
val Array(benTrainDF, benEvalDF) = final_df_all.randomSplit(Array(0.8, 0.02),seed = 12345)

benTrainDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: float, features: array<float> ... 4 more fields]
benEvalDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: float, features: array<float> ... 4 more fields]


In [15]:
benTrainDF.count()

res17: Long = 10728


In [16]:
benEvalDF.count()

res18: Long = 272


In [17]:
val EvalDF = benEvalDF.union(van)

EvalDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: float, features: array<float> ... 4 more fields]


In [18]:
EvalDF.where($"target"===1).count()

res19: Long = 1023


In [20]:
EvalDF.where($"target"===0).count()

res21: Long = 249


In [32]:
val ben_sim_ts_td_meta = (fs.createTrainingDataset()
                .name("ben_td")
                .description("Ben simulated Time Series Training Dataset")
                .dataFormat(DataFormat.TFRECORD)     
                .statisticsEnabled(false)          
                .version(1)
                .build())
ben_sim_ts_td_meta.save(benTrainDF)

ben_sim_ts_td_meta: com.logicalclocks.hsfs.TrainingDataset = com.logicalclocks.hsfs.TrainingDataset@60349314


In [34]:
val eval_sim_ts_td_meta = (fs.createTrainingDataset()
                .name("eval_td")
                .description("Simulated Time Series Training Dataset for evaluation")
                .dataFormat(DataFormat.TFRECORD)     
                .statisticsEnabled(false)          
                .version(1)
                .build())
eval_sim_ts_td_meta.save(EvalDF)

eval_sim_ts_td_meta: com.logicalclocks.hsfs.TrainingDataset = com.logicalclocks.hsfs.TrainingDataset@243290ea
